# Establishing a Connection - Alice
## Role: Inviter

In this notebook we'll be initiating the aries DID Exchange protocol. Before an agent can begin to interact with another agent they must engage in this protocol resulting in the exchange of Decentralised Identifiers (DIDs) and corresponding DID Documents. The information within these documents can then be used to establish a secure communication channel between the parties.

For a detailed description of this protocol it is highly recommened to read this [aries-rfc]((https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange).

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's notebook

<b>Carry on in [Bob's notebook](http://localhost:8889/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>
    
11. Accept Request for Connection
12. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [10]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [11]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)


### 3. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [12]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([], defaults=True)

Task exception was never retrieved
future: <Task finished coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:77> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 83, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.7/site-packages/aiohttp/web_runner.py", line 104, in start
    reuse_port=self._reuse_port)
  File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 1385, in create_server
    % (sa, err.strerror.lower())) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [8]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite)

ClientConnectorError: Cannot connect to host alice-agent:8021 ssl:default [Connect call failed ('172.19.0.4', 8021)]

Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site

### 5. Copy the invitation output from 4 and move over to the [Bob notebook](http://localhost:8889/notebooks/Part%203%20-%20Establishing%20a%20Connection.ipynb) 

### 11. Accept Request for Connection

In [5]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

True
ACCEPT REQUEST
{'initiator': 'self', 'state': 'response', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'NC8XUB5EicuUUqJACvSUdR', 'accept': 'manual', 'invitation_key': 'AGQrdCLtBbPPs5vX8K8RJrns9UyZLzvW5rvszNWrzq1V', 'created_at': '2020-09-16 15:08:58.841738Z', 'their_did': 'KbykKreVe4J8n5gizLmq4Z', 'routing_state': 'none', 'connection_id': 'c62331ca-bc95-4f7a-93dd-043f501cb21a', 'updated_at': '2020-09-16 15:11:48.372610Z'}
state response


### 12. Send Trust Ping to activate the conneciton

In [6]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': '694105cd-305c-42ec-8c32-bf6ac5fcb868'}


####  12.1 Check if connection is active

In [7]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

{'initiator': 'self', 'state': 'active', 'their_label': 'Bob', 'invitation_mode': 'once', 'my_did': 'NC8XUB5EicuUUqJACvSUdR', 'accept': 'manual', 'invitation_key': 'AGQrdCLtBbPPs5vX8K8RJrns9UyZLzvW5rvszNWrzq1V', 'created_at': '2020-09-16 15:08:58.841738Z', 'their_did': 'KbykKreVe4J8n5gizLmq4Z', 'routing_state': 'none', 'connection_id': 'c62331ca-bc95-4f7a-93dd-043f501cb21a', 'updated_at': '2020-09-16 15:11:53.300639Z'}
Is Active? active


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [9]:
response = await agent_controller.terminate()
print(response)

None
